In [ ]:
import copy
from datetime import datetime
from itertools import product
from importlib import reload
from IPython import extract_module_locals
import pathlib

import numpy as np
import matplotlib.pyplot as plt

from exactdiag.logging_utils import setup_logging
from exactdiag.tJ_spin_half_ladder import api
from exactdiag.tJ_spin_half_ladder import configs


main_folder = pathlib.Path(extract_module_locals()[1]["__vsc_ipynb_file__"]).parents[3]
%cd {main_folder}

from examples.tJ_spin_half_ladder.inter_ladder_conductivity import functions as fs  # noqa: E402 - I could not make it work as a relative import.

reload(fs)

setup_logging()  # The logs seem to show up somewhat inconsistently.

In [ ]:
# Here, we calculate the inter-ladder conductivity as spectral function convolution.
# This assumes that the adjecent ladders are uncorrelated.

config = configs.Config(
    hamiltonian={
        "num_rungs": 10,
        "num_holes": 4,
        "weights": {"tl": 1, "tr": 1, "jl": 1.5, "jr": 1.5},
        "num_threads": 12,
        "symmetry_qs": {"leg": 0, "rung": 0},
        # The symmetry of q -> -q that comes with k=[0,0] is assumed later.
    },
    eigenpair={"num_eigenpairs": 5},
    spectrum={
        "name": "spectral_function",
        "operator_symmetry_qs": None,
        "omega_max": 4,  # The current convolution requires symmetric bounds.
        "omega_min": -4,
        "omega_steps": 1000,
        "num_lanczos_vecs": 250,
        "broadening": 0.05,
    },
)  # NOTE: We may mutate the config as we go.

In [ ]:
fig_folder = config.hamiltonian.get_figures_folder()

# Plot a series with one changing weight both separately and as a single figure.
trs = [0.1, 0.15, 0.2, 0.25, 0.4, 0.6, 0.8, 1]
for tr in trs:
    config.hamiltonian.weights.tr = tr
    ws, sigma_2357, ts = fs.get_conductivity_contributions(config)
    fs.plot_interladder_conductivity(config, ws, sigma_2357, ts, folder=fig_folder)
fs.multiple_plots(config, trs, 1, r"t_\perp", lines_per_plot=4, folder=fig_folder)


# Plot isotropic case with energy normed by J
J_to_ts = np.linspace(0.1, 1.5, 8)
for J_to_t in J_to_ts:
    config.hamiltonian.weights.jl = J_to_t
    config.hamiltonian.weights.jr = J_to_t
    ws, sigma_2357, ts = fs.get_conductivity_contributions(config)
    plt.plot(ws / J_to_t, sigma_2357, label=f"$J/t={J_to_t:g}$")
plt.ylabel(r"$\sigma(\omega)$ [a.u.]")
plt.xlabel("frequency [J]")
plt.xlim([0, 2])
plt.ylim([0, None])
plt.legend(loc="upper left", bbox_to_anchor=(1.05, 1.05))
fig_path = fig_folder / f"interladder_sigma_isotropic_inJ_{J_to_ts[0]:g},{J_to_ts[-1]:g}.pdf"
plt.savefig(fig_path, bbox_inches='tight')
plt.show()


J_to_t = 5
config.hamiltonian.weights.jl = J_to_t
config.hamiltonian.weights.jr = J_to_t

ws, sigma_2357, ts = fs.get_conductivity_contributions(config)
plt.plot(ws / J_to_t, sigma_2357, label=f"$J/t={J_to_t:g}$")
plt.ylabel(r"$\sigma(\omega)$ [a.u.]")
plt.xlabel("frequency [J]")
plt.xlim([0, None])
# plt.ylim([0,None])
plt.legend(loc="upper left", bbox_to_anchor=(1.05, 1.05))
fig_path = fig_folder / f"interladder_sigma_isotropic_inJ_J={J_to_t}t.pdf"
plt.savefig(fig_path, bbox_inches='tight')
plt.show()